# Tutorial Exercise: Yelp reviews

## Introduction

This exercise uses a small subset of the data from Kaggle's [Yelp Business Rating Prediction](https://www.kaggle.com/c/yelp-recsys-2013) competition.

**Description of the data:**

- **`yelp.csv`** contains the dataset. It is stored in the repository (in the **`data`** directory), so there is no need to download anything from the Kaggle website.
- Each observation (row) in this dataset is a review of a particular business by a particular user.
- The **stars** column is the number of stars (1 through 5) assigned by the reviewer to the business. (Higher stars is better.) In other words, it is the rating of the business by the person who wrote the review.
- The **text** column is the text of the review.

**Goal:** Predict the star rating of a review using **only** the review text.

**Tip:** After each task, I recommend that you check the shape and the contents of your objects, to confirm that they match your expectations.

In [1]:
import pandas as pd
import numpy as np

## Task 1

Read **`yelp.csv`** into a pandas DataFrame and examine it.

In [3]:
df = pd.read_csv('data/yelp.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  10000 non-null  object
 1   date         10000 non-null  object
 2   review_id    10000 non-null  object
 3   stars        10000 non-null  int64 
 4   text         10000 non-null  object
 5   type         10000 non-null  object
 6   user_id      10000 non-null  object
 7   cool         10000 non-null  int64 
 8   useful       10000 non-null  int64 
 9   funny        10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB
None


## Task 2

Create a new DataFrame that only contains the **5-star** and **1-star** reviews.

- **Hint:** [How do I apply multiple filter criteria to a pandas DataFrame?](http://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb#9.-How-do-I-apply-multiple-filter-criteria-to-a-pandas-DataFrame%3F-%28video%29) explains how to do this.

In [7]:
df_fil = df[(df['stars']==1)|(df['stars']==5)]
df_fil.shape

(4086, 10)

## Task 3

Define X and y from the new DataFrame, and then split X and y into training and testing sets, using the **review text** as the only feature and the **star rating** as the response.

- **Hint:** Keep in mind that X should be a pandas Series (not a DataFrame), since we will pass it to CountVectorizer in the task that follows.

In [8]:
X = df_fil['text']
y = df_fil['stars']
print(type(X))
print(type(y))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Task 4

Use CountVectorizer to create **document-term matrices** from X_train and X_test.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)
X_train_vec = vect.transform(X_train)
X_test_vec = vect.transform(X_test)
print(X_train_vec.shape)
print(X_test_vec.shape)

(3064, 16825)
(1022, 16825)


## Task 5

Use multinomial Naive Bayes to **predict the star rating** for the reviews in the testing set, and then **calculate the accuracy** and **print the confusion matrix**.

- **Hint:** [Evaluating a classification model](https://github.com/justmarkham/scikit-learn-videos/blob/master/09_classification_metrics.ipynb) explains how to interpret both classification accuracy and the confusion matrix.

In [12]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
y_pred_class = nb.predict(X_test_vec)

In [14]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

0.9187866927592955
[[126  58]
 [ 25 813]]


## Task 6 (Challenge)

Calculate the **null accuracy**, which is the classification accuracy that could be achieved by always predicting the most frequent class.

- **Hint:** [Evaluating a classification model](https://github.com/justmarkham/scikit-learn-videos/blob/master/09_classification_metrics.ipynb) explains null accuracy and demonstrates two ways to calculate it, though only one of those ways will work in this case. Alternatively, you can come up with your own method to calculate null accuracy!

In [17]:
y_test.value_counts()

stars
5    838
1    184
Name: count, dtype: int64

In [29]:
y_test[y_test==5].count()/y_test.count()

0.8199608610567515

## Task 7 (Challenge)

Browse through the review text of some of the **false positives** and **false negatives**. Based on your knowledge of how Naive Bayes works, do you have any ideas about why the model is incorrectly classifying these reviews?

- **Hint:** [Evaluating a classification model](https://github.com/justmarkham/scikit-learn-videos/blob/master/09_classification_metrics.ipynb) explains the definitions of "false positives" and "false negatives".
- **Hint:** Think about what a false positive means in this context, and what a false negative means in this context. What has scikit-learn defined as the "positive class"?

In [36]:
#printing false positives
X_test[y_pred_class > y_test].head()

2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
Name: text, dtype: object

In [37]:
#printing false negatives
X_test[y_pred_class < y_test].head()

7148    I now consider myself an Arizonian. If you dri...
4963    This is by far my favourite department store, ...
6318    Since I have ranted recently on poor customer ...
380     This is a must try for any Mani Pedi fan. I us...
5565    I`ve had work done by this shop a few times th...
Name: text, dtype: object

## Task 8 (Challenge)

Calculate which 10 tokens are the most predictive of **5-star reviews**, and which 10 tokens are the most predictive of **1-star reviews**.

- **Hint:** Naive Bayes automatically counts the number of times each token appears in each class, as well as the number of observations in each class. You can access these counts via the `feature_count_` and `class_count_` attributes of the Naive Bayes model object.

In [42]:
# create a DataFrame of tokens with their separate positive and negative counts
tokens = pd.DataFrame({'token':vect.get_feature_names_out(), 'pos':nb.feature_count_[1,:], 'neg':nb.feature_count_[0,:]}).set_index('token')
tokens.head()

,pos,neg
token,,
00,39.0,26.0
000,5.0,4.0
00a,0.0,1.0
00am,2.0,3.0
00pm,4.0,1.0


In [47]:
#add 1 to calculate %
tokens['pos']=tokens['pos']+1
tokens['neg']=tokens['neg']+1

In [50]:
tokens['pos_per'] = tokens['pos']/nb.class_count_[1]
tokens['neg_per'] = tokens['neg']/nb.class_count_[0]

In [51]:
#print most predictive of positive
tokens.loc[:,'pos_per'].sort_values(ascending=False).head(10)

token
the    5.605842
and    4.173669
to     2.534614
of     1.823930
is     1.757103
it     1.704682
was    1.414566
in     1.395358
for    1.326931
you    1.161665
Name: pos_per, dtype: float64

In [52]:
#print most predictive of positive
tokens.loc[:,'neg'].sort_values(ascending=False).head(10)

token
the     4176.0
and     2660.0
to      2410.0
was     1485.0
it      1329.0
of      1295.0
for      971.0
that     959.0
in       942.0
my       842.0
Name: neg, dtype: float64

In [53]:
tokens['five_ratio'] = tokens['pos']/tokens['neg']
tokens.loc[:,'five_ratio'].sort_values(ascending = False).head(10)

token
fantastic      96.500000
perfect        81.666667
yum            62.000000
favorite       49.285714
outstanding    49.000000
brunch         42.000000
gem            40.000000
pasty          39.000000
mozzarella     39.000000
amazing        38.583333
Name: five_ratio, dtype: float64

In [54]:
tokens.loc[:,'five_ratio'].sort_values(ascending = False).tail(10)

token
fuse              0.125000
boca              0.125000
ugh               0.117647
unacceptable      0.111111
unprofessional    0.111111
acknowledge       0.111111
filthy            0.090909
disgusting        0.083333
refused           0.071429
staffperson       0.058824
Name: five_ratio, dtype: float64

## Task 9 (Challenge)

Up to this point, we have framed this as a **binary classification problem** by only considering the 5-star and 1-star reviews. Now, let's repeat the model building process using all reviews, which makes this a **5-class classification problem**.

Here are the steps:

- Define X and y using the original DataFrame. (y should contain 5 different classes.)
- Split X and y into training and testing sets.
- Create document-term matrices using CountVectorizer.
- Calculate the testing accuracy of a Multinomial Naive Bayes model.
- Compare the testing accuracy with the null accuracy, and comment on the results.
- Print the confusion matrix, and comment on the results. (This [Stack Overflow answer](http://stackoverflow.com/a/30748053/1636598) explains how to read a multi-class confusion matrix.)
- Print the [classification report](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-report), and comment on the results. If you are unfamiliar with the terminology it uses, research the terms, and then try to figure out how to calculate these metrics manually from the confusion matrix!

In [65]:
#first we create X and y from the original dataset (including all ratings). Also check the type
X = df['text']
y = df['stars']
print(type(X))
print(type(y))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [66]:
#Test/Train split of the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [67]:
#Vectorizer for the reviews using standard settings, 
vect = CountVectorizer()
vect.fit(X_train)
X_train_vec = vect.transform(X_train)
X_test_vec = vect.transform(X_test)

In [68]:
#Use a multinomual Naive Bayes model to classify the reviews and predict number of stars
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
y_pred_class = nb.predict(X_test_vec)

In [69]:
#Print accuracy
print(metrics.accuracy_score(y_test, y_pred_class))

0.4712


In [74]:
#Print null accuracy
y.value_counts().iloc[0]/y.count()

0.3526

In [62]:
print(metrics.confusion_matrix(y_test, y_pred_class))

[[ 55  14  24  65  27]
 [ 28  16  41 122  27]
 [  5   7  35 281  37]
 [  7   0  16 629 232]
 [  6   4   6 373 443]]


In [64]:
print(metrics.classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           1       0.54      0.30      0.38       185
           2       0.39      0.07      0.12       234
           3       0.29      0.10      0.14       365
           4       0.43      0.71      0.53       884
           5       0.58      0.53      0.55       832

    accuracy                           0.47      2500
   macro avg       0.45      0.34      0.35      2500
weighted avg       0.46      0.47      0.43      2500

